In [123]:
%load_ext autoreload
%autoreload 2

from scribbles_creator import *
from PIL import Image
import napari
import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create scribble annotation from an image mask of the cell pose dataset

In [125]:
def create_cellpose_scribble(folder_path, img_num, min_perc=0.2, sq_scaling=400, save_res=False, show_res=False):

    # Read the ground truth as an image
    image_path = f"{folder_path}/{str(img_num).zfill(3)}"
    ground_truth = np.array(Image.open(image_path + "_masks.png"))
    # Summarize all non-background classes into one class (we are testing semantic segmentation not instance segmentation)
    ground_truth[ground_truth>0] = 2
    # Set the background class to 1 (since the scribble annotation assumes class 0 to represent non-annotated pixels)
    ground_truth[ground_truth==0] = 1

    # Define parameters for the scribble annotation
    min_perc_per_mode = min_perc / 3
    scribble_width = 1
    sq_size = (ground_truth.shape[0] * ground_truth.shape[1] // sq_scaling) ** 0.5
    sq_size = int(sq_size)
    annot_pix_range = (sq_size//2, sq_size*2)
    mode = "all"

    # Generate the scribble annotation for the ground truth
    scribbles = create_scribble(ground_truth, scribble_width=scribble_width, sk_min_perc=min_perc_per_mode, sq_size=sq_size, sq_pix_range=annot_pix_range, lines_min_perc=min_perc_per_mode, line_pix_range=annot_pix_range, mode=mode)
    perc_labelled = np.sum(scribbles>0) / (scribbles.shape[0] * scribbles.shape[1]) * 100

    if save_res:
        # Save the scribble annotation as an image
        scribble_img = Image.fromarray(scribbles)
        scribble_img.save(image_path + f"_scribbles_{min_perc}.png")
        # If the ground truth file does not exist, save it as well
        gt_file = image_path + "_ground_truth.png"
        if not os.path.exists(gt_file):
            gt_img = Image.fromarray(ground_truth)
            gt_img.save(gt_file)

    if show_res:
        # Show the ground truth and the scribble annotation
        v = napari.Viewer()
        image_path = f"{folder_path}/{str(im_num).zfill(3)}"
        image = np.array(Image.open(image_path + "_img.png"))
        ground_truth = np.array(Image.open(image_path + "_masks.png"))
        v.add_image(image)
        v.add_labels(ground_truth)
        v.add_labels(scribbles)

    return scribbles, perc_labelled

In [126]:
folder_path = "./imgs/train"
for im_num in range(1, 2):
    # im_num = 332
    scribbles, perc_labelled = create_cellpose_scribble(folder_path, im_num, min_perc=0.2, show_res=False, save_res=True)
    print(f"Percentage of labelled pixels: {perc_labelled:.2f}%")

Percentage of labelled pixels: 0.23%


## Save or show the scribbles and ground truth

In [129]:
# Show the image, ground truth and the scribble annotation in a napari viewer
v = napari.Viewer()
image_path = f"{folder_path}/{str(im_num).zfill(3)}"
image = np.array(Image.open(image_path + "_img.png"))
ground_truth = np.array(Image.open(image_path + "_ground_truth.png"))
scribbles = np.array(Image.open(image_path + "_scribbles_0.2.png"))
v.add_image(image)
v.add_labels(ground_truth)
v.add_labels(scribbles)

<Labels layer 'scribbles' at 0x21f8c2455e0>

In [ ]:
# Save the scribbles and ground truth image to a file (first, convert the numpy arrays to images)
scribbles_image = Image.fromarray(scribbles)
ground_truth_image = Image.fromarray(ground_truth)
scribbles_image.save(image_path + '_scribbles.png')
ground_truth_image.save(image_path + '_ground_truth.png')

## When we have performed segmentation in Napari, we can calculate its accuracy

In [ ]:
segmentation = v.layers['segmentation'].data
acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
print(f"Accuracy (pixels correctly segmented): {acc}%")